#01 First - Webscrap and create a dataframe

In [13]:

#imports
import numpy as np # para a manipulação de vetores
import pandas as pd # para a analise de dados
import json # para manipulação de .json

# para transformar um endereço em latitude e longitude
from geopy.geocoders import Nominatim 
# manipulação de requests
import requests 
# para transformar um .json em um dataframe pandas
from pandas.io.json import json_normalize 
# Matplot para criação dos graficos
import matplotlib.cm as cm
import matplotlib.colors as colors
# Sklearning usado para machine learning Kmeans para agrupamento
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
#para a geranção de mapas
import folium
from bs4 import BeautifulSoup
import lxml
#imput output
import io
#importar geocoder
import geocoder 



#remover limitador para visualizacao
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print('ok')

ok


In [14]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')
#print(soup.prettify())


In [15]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

#print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
#________________________________________________________________________________________________
#Delete rows where Borough is 'Not assigned'
df = df[df.Borough!= "Not assigned"]

#Combine all neighbourhoods where postcode and Borough are the same
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()
#if Neighbourhood is not assigned, replace by borough name
for index, row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']

# define the dataframe columns
column_names = ['PostalCode','Borough', 'Neighborhood']

# instantiate the dataframe
df.columns = column_names

print(df.to_markdown())

|     | PostalCode   | Borough                | Neighborhood                                                                                                                           |
|----:|:-------------|:-----------------------|:---------------------------------------------------------------------------------------------------------------------------------------|
|   0 | M1B          | Scarborough            | Malvern, Rouge                                                                                                                         |
|   1 | M1C          | Scarborough            | Rouge Hill, Port Union, Highland Creek                                                                                                 |
|   2 | M1E          | Scarborough            | Guildwood, Morningside, West Hill                                                                                                      |
|   3 | M1G          | Scarborough            | Woburn                     

#02 Second Task  - Insert longitude and lattitute in dataframe

In [22]:
url="https://cocl.us/Geospatial_data"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

dfc = df.join(c.set_index('Postal Code'), on='PostalCode')
dfc.head()
print(dfc.to_markdown())

|     | PostalCode   | Borough                | Neighborhood                                                                                                                           |   Latitude |   Longitude |
|----:|:-------------|:-----------------------|:---------------------------------------------------------------------------------------------------------------------------------------|-----------:|------------:|
|   0 | M1B          | Scarborough            | Malvern, Rouge                                                                                                                         |    43.8067 |    -79.1944 |
|   1 | M1C          | Scarborough            | Rouge Hill, Port Union, Highland Creek                                                                                                 |    43.7845 |    -79.1605 |
|   2 | M1E          | Scarborough            | Guildwood, Morningside, West Hill                                                                       

# 03 Third task- Show neighborhoods in Toronto

In [26]:
address = 'Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Toronto coordinates {}, {}.'.format(latitude, longitude))

Toronto coordinates 43.6534817, -79.3839347.


In [25]:
#map
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

map_toronto

In [27]:
#insert markers

for lat, lng, borough, neighborhood in zip(dfc['Latitude'], dfc['Longitude'], dfc['Borough'], dfc['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [34]:
#Fousquare
CLIENT_ID = 'BNIAVGNS2GGN3WW4RLKOA2FTWFWTLT3WBFMN1ZU2F1U2FBD5' # your Foursquare ID
CLIENT_SECRET = 'SJJFOWFEIGF5KN2IWB14G5ANP4NHB5QOCKZTBOQLNCINVLOL' # your Foursquare Secret
ACCESS_TOKEN = 'DSAIWWVJH3TYFYFXICF3YCAG0KOPW2MQHUE5F0NSNWMXHRGV' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
radius = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


#Send the GET request and examine the results
results = requests.get(url).json()
results

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

Your credentails:
CLIENT_ID: BNIAVGNS2GGN3WW4RLKOA2FTWFWTLT3WBFMN1ZU2F1U2FBD5
CLIENT_SECRET:SJJFOWFEIGF5KN2IWB14G5ANP4NHB5QOCKZTBOQLNCINVLOL


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,5b193c42598e64002ca79b96,City of Toronto Civic Innovation Office,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",v-1621189304,False,100 Queen St W,43.653454,-79.383952,"[{'label': 'display', 'lat': 43.653454, 'lng':...",3,M5H 2N2,CA,Toronto,ON,Canada,"[100 Queen St W, Toronto ON M5H 2N2, Canada]",NaN
1,4ad4c05ef964a5208ff620e3,Toronto City Hall,"[{'id': '4bf58dd8d48988d129941735', 'name': 'C...",v-1621189304,False,100 Queen St. W.,43.653140,-79.383967,"[{'label': 'display', 'lat': 43.65313989695342...",38,M5H 2N2,CA,Toronto,ON,Canada,"[100 Queen St. W. (at Bay St.), Toronto ON M5H...",at Bay St.
2,4c41d47cd691c9b6fa4c8d0a,Kew Gardens Playground,"[{'id': '4bf58dd8d48988d1e7941735', 'name': 'P...",v-1621189304,False,Queen street and belfair ave,43.653225,-79.383185,"[{'label': 'display', 'lat': 43.65322456230742...",66,NaN,CA,Toronto,ON,Canada,"[Queen street and belfair ave, Toronto ON, Can...",NaN
3,4c0121fd9a950f47fa9208c6,City Hall Podium Green Roof,"[{'id': '4bf58dd8d48988d15a941735', 'name': 'G...",v-1621189304,False,100 Queen Street West,43.653504,-79.383866,"[{'label': 'display', 'lat': 43.65350358617817...",6,NaN,CA,Toronto,ON,Canada,"[100 Queen Street West, Toronto ON, Canada]",NaN
4,4b06ef4cf964a52072f322e3,Cafe On The Square,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",v-1621189304,False,100 Queen St W,43.652633,-79.383361,"[{'label': 'display', 'lat': 43.65263298456729...",105,M5H 2N2,CA,Toronto,ON,Canada,"[100 Queen St W, Toronto ON M5H 2N2, Canada]",NaN


In [ ]:
# 04 Forth task- Make analysis

In [35]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,City of Toronto Civic Innovation Office,City Hall,100 Queen St W,43.653454,-79.383952,"[{'label': 'display', 'lat': 43.653454, 'lng':...",3,M5H 2N2,CA,Toronto,ON,Canada,"[100 Queen St W, Toronto ON M5H 2N2, Canada]",NaN,5b193c42598e64002ca79b96
1,Toronto City Hall,City Hall,100 Queen St. W.,43.653140,-79.383967,"[{'label': 'display', 'lat': 43.65313989695342...",38,M5H 2N2,CA,Toronto,ON,Canada,"[100 Queen St. W. (at Bay St.), Toronto ON M5H...",at Bay St.,4ad4c05ef964a5208ff620e3
2,Kew Gardens Playground,Playground,Queen street and belfair ave,43.653225,-79.383185,"[{'label': 'display', 'lat': 43.65322456230742...",66,NaN,CA,Toronto,ON,Canada,"[Queen street and belfair ave, Toronto ON, Can...",NaN,4c41d47cd691c9b6fa4c8d0a
3,City Hall Podium Green Roof,Garden,100 Queen Street West,43.653504,-79.383866,"[{'label': 'display', 'lat': 43.65350358617817...",6,NaN,CA,Toronto,ON,Canada,"[100 Queen Street West, Toronto ON, Canada]",NaN,4c0121fd9a950f47fa9208c6
4,Cafe On The Square,Restaurant,100 Queen St W,43.652633,-79.383361,"[{'label': 'display', 'lat': 43.65263298456729...",105,M5H 2N2,CA,Toronto,ON,Canada,"[100 Queen St W, Toronto ON M5H 2N2, Canada]",NaN,4b06ef4cf964a52072f322e3


In [36]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,City of Toronto Civic Innovation Office,City Hall,100 Queen St W,43.653454,-79.383952,"[{'label': 'display', 'lat': 43.653454, 'lng':...",3,M5H 2N2,CA,Toronto,ON,Canada,"[100 Queen St W, Toronto ON M5H 2N2, Canada]",NaN,5b193c42598e64002ca79b96
1,Toronto City Hall,City Hall,100 Queen St. W.,43.653140,-79.383967,"[{'label': 'display', 'lat': 43.65313989695342...",38,M5H 2N2,CA,Toronto,ON,Canada,"[100 Queen St. W. (at Bay St.), Toronto ON M5H...",at Bay St.,4ad4c05ef964a5208ff620e3
2,Kew Gardens Playground,Playground,Queen street and belfair ave,43.653225,-79.383185,"[{'label': 'display', 'lat': 43.65322456230742...",66,NaN,CA,Toronto,ON,Canada,"[Queen street and belfair ave, Toronto ON, Can...",NaN,4c41d47cd691c9b6fa4c8d0a
3,City Hall Podium Green Roof,Garden,100 Queen Street West,43.653504,-79.383866,"[{'label': 'display', 'lat': 43.65350358617817...",6,NaN,CA,Toronto,ON,Canada,"[100 Queen Street West, Toronto ON, Canada]",NaN,4c0121fd9a950f47fa9208c6
4,Cafe On The Square,Restaurant,100 Queen St W,43.652633,-79.383361,"[{'label': 'display', 'lat': 43.65263298456729...",105,M5H 2N2,CA,Toronto,ON,Canada,"[100 Queen St W, Toronto ON M5H 2N2, Canada]",NaN,4b06ef4cf964a52072f322e3


In [39]:
#Create function to know how many venues there are in Toronto

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)




#List of Neighborhood that have venues in Toronto
toronto_venues = getNearbyVenues(names=dfc['Neighborhood'],
                                   latitudes=dfc['Latitude'],
                                   longitudes=dfc['Longitude']
                                  )

toronto_venues.head()

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
The Danforth  East
The Danforth West, Riverdale


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Malvern, Rouge",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Great Shine Window Cleaning,43.783145,-79.157431,Home Service
3,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank


In [45]:

#Number of venues per neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",6,6,6,6,6,6
"Bathurst Manor, Wilson Heights, Downsview North",22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",26,26,26,26,26,26
Berczy Park,30,30,30,30,30,30
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17


In [47]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 239 uniques categories.


In [50]:

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
manhattan_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Repair Shop,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.200000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [51]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Breakfast Spot   0.2
1                     Lounge   0.2
2             Clothing Store   0.2
3  Latin American Restaurant   0.2
4               Skating Rink   0.2


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.33
1  Sandwich Place  0.17
2     Coffee Shop  0.17
3             Pub  0.17
4             Gym  0.17


----Bathurst Manor, Wilson Heights, Downsview North----
                       venue  freq
0                Coffee Shop  0.09
1                       Bank  0.09
2                   Pharmacy  0.05
3                Gas Station  0.05
4  Middle Eastern Restaurant  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1   Chinese Restaurant  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4          Music Venue  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.12
1         Coffee Shop  0.08

In [52]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]



num_top_venues = 4

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
0,Agincourt,Latin American Restaurant,Clothing Store,Skating Rink,Breakfast Spot
1,"Alderwood, Long Branch",Pizza Place,Sandwich Place,Pub,Coffee Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Park,Pharmacy
3,Bayview Village,Chinese Restaurant,Bank,Japanese Restaurant,Café
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Sandwich Place,Coffee Shop,Restaurant


In [53]:

# set number of clusters
kclusters = 2

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]



# add clustering labels
neighborhoods_venues_sorted.insert(0, 'ClusterLabels', kmeans.labels_)

toronto_merged = dfc

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged = toronto_merged.dropna()


toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,0.0,Fast Food Restaurant,Print Shop,Yoga Studio,Curling Ice
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,0.0,Bar,Home Service,Yoga Studio,Dance Studio
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Restaurant,Medical Center,Rental Car Location,Intersection
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Soccer Field,Korean BBQ Restaurant,Dance Studio
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Fried Chicken Joint,Caribbean Restaurant,Bank,Athletics & Sports


In [54]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['ClusterLabels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [55]:
toronto_merged.loc[toronto_merged['ClusterLabels'] ==1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,ClusterLabels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
14,Scarborough,1.0,Playground,Park,Intersection,Yoga Studio
21,North York,1.0,Park,Yoga Studio,Curling Ice,Eastern European Restaurant
23,North York,1.0,Park,Convenience Store,Yoga Studio,Dance Studio
25,North York,1.0,Park,Food & Drink Shop,Yoga Studio,Dance Studio
50,Downtown Toronto,1.0,Park,Trail,Playground,Yoga Studio
74,York,1.0,Park,Women's Store,Pool,Curling Ice
98,York,1.0,Park,Convenience Store,Yoga Studio,Dance Studio
100,Etobicoke,1.0,Park,Sandwich Place,Mobile Phone Shop,Yoga Studio
